In [2]:
# Ignore SQLITE warnings related to Decimal numbers in the Chinook database
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Import Dependencies
### BEGIN SOLUTION
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func
from sqlalchemy import inspect
### END SOLUTION

In [4]:
import pandas as pd
import json
import numpy as np

In [ ]:
##################################################
# Flask Setup
##################################################
app = Flask(__name__)

In [5]:
# Create an engine for the belly_button_biodiversity.sqlite database
### BEGIN SOLUTION
engine = create_engine("sqlite:///belly_button_biodiversity.sqlite", echo=False)
### END SOLUTION

In [6]:
# Reflect Database into ORM classes
### BEGIN SOLUTION
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()
### END SOLUTION

['otu', 'samples', 'samples_metadata']

In [7]:
#Get References for each table
Otu = Base.classes.otu
Samples = Base.classes.samples
Samples_metadata = Base.classes.samples_metadata

In [12]:
inspector = inspect(engine)
samplesall = inspector.get_columns('samples')
session = Session(engine)

In [109]:
# Other ways of doing this
# Converting using pandas and sql statement
# stmt = session.query(Samples).statement
# df = pd.read_sql_query(stmt, session.bind)
# df.set_index('otu_id', inplace=True)
# df.columns
# # Get the sample list which is the column names in Samples table 
# samples_column_list = Samples.__table__.columns.keys()
# # First element in the samples_column_list is 'otu_id' so sample list is from 2nd element to the last element
# sample_list = samples_column_list[1:]
# print(sample_list)
# return jsonify(sample_list)

#### Getting Sample Names

In [14]:
# Converting using pandas
samplesall_df = pd.DataFrame(samplesall)
# samplenames = samplesall_df.loc[samplesall_df["name"] != 'otu_id',]
samplenamesdf = pd.DataFrame(samplesall_df['name'])
samplenamesdf.set_index('name',inplace=True)
# json.dumps(list(samplesall_df['name'][1:]))

#### OTU Descriptions

In [15]:
results = session.query(Otu.lowest_taxonomic_unit_found).all()
# json.dumps(list(np.ravel(results)))

#### Metadata for sample

In [16]:
sample = 'BB_940'
sampleip = sample[3:]
metadata = session.query(Samples_metadata).filter(Samples_metadata.SAMPLEID == sampleip).first()
# print(metadata.AGE)
metadata_dict={"AGE":metadata.AGE,
            "GENDER": metadata.GENDER,
            "ETHNICITY": metadata.ETHNICITY,
            "BBTYPE": metadata.BBTYPE,
            "LOCATION": metadata.LOCATION,
            "SAMPLEID": metadata.SAMPLEID}
# print(metadata_dict)
# json.dumps(metadata_dict)

#### Washing Frequency

In [17]:
sample = 'BB_940'
sampleip = sample[3:]
metadata = session.query(Samples_metadata).filter(Samples_metadata.SAMPLEID == sampleip).first()
print(metadata.WFREQ)

2


#### OTU ids and samples for each sample

In [18]:
sample = 'BB_940'
stmt = session.query(Samples).statement
df = pd.read_sql_query(stmt, session.bind)
df = df[df[sample] > 1]
df = df.sort_values(by=sample, ascending=0)
# Format the data to send as json
data = [{
        "otu_ids": df[sample].index.values.tolist(),
        "sample_values": df[sample].values.tolist()
    }]
# json.dumps(data)

In [19]:
#Without using Pandas 
# samples_result = session.query(Samples.otu_id,getattr(Samples,sample)).order_by(getattr(Samples,sample).desc()).all()
# otu_id_list = [i[0] for i in samples_result]
# sample_values_list= [i[1] for i in samples_result]
# result={"otu_ids":otu_id_list, "sample_values": sample_values_list}
# json.dumps([result])